# RAG Document Q&A - Exploration Notebook

This notebook is for testing API connections and experimenting with the RAG system components.

## 1. Environment Setup & Validation

In [ ]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()

print("Environment Check:")
print(f"  OpenRouter API Key: {'✓ Set' if os.getenv('OPENROUTER_API_KEY') else '✗ Missing'}")
print(f"  Groq API Key: {'✓ Set' if os.getenv('GROQ_API_KEY') else '✗ Missing'}")
print(f"  Langfuse Keys: {'✓ Set' if os.getenv('LANGFUSE_SECRET_KEY') else '✗ Missing'}")

In [ ]:
# Test configuration loading
from src.config import settings

print("Configuration:")
print(f"  Active Provider: {settings.get_active_llm_provider()}")
print(f"  LLM Model: {settings.llm_model}")
print(f"  Embedding Model: {settings.embedding_model}")
print(f"  Chunk Size: {settings.chunk_size}")
print(f"  Chunk Overlap: {settings.chunk_overlap}")
print(f"  Retrieval K: {settings.retrieval_k}")

## 2. Test LLM Connection

In [ ]:
from src.chain import get_llm

# Create LLM instance
llm = get_llm()

# Test with a simple message
response = llm.invoke("Hello! Say 'RAG system ready!' if you can hear me.")
print(f"LLM Response: {response.content}")

In [ ]:
# Test streaming
print("Streaming test:")
for chunk in llm.stream("Count from 1 to 5, one number per line."):
    print(chunk.content, end="", flush=True)
print()

## 3. Test Embeddings

In [ ]:
from src.embeddings import get_embeddings, embed_query

# Get embeddings model
embeddings = get_embeddings()
print(f"Embeddings Model: {embeddings.model}")

# Test embedding a query
test_text = "What is RAG and how does it work?"
embedding = embed_query(test_text)
print(f"\nEmbedding dimension: {len(embedding)}")
print(f"First 5 values: {embedding[:5]}")

## 4. Test Document Loading

In [ ]:
from src.loaders import load_document
from src.chunker import chunk_documents

# Create a test document
test_content = """# RAG System Overview

Retrieval-Augmented Generation (RAG) is a technique that enhances LLM responses
by retrieving relevant information from a knowledge base.

## How It Works

1. Documents are loaded and split into chunks
2. Chunks are converted to vector embeddings
3. Embeddings are stored in a vector database
4. User queries are converted to embeddings
5. Similar chunks are retrieved
6. LLM generates response using retrieved context

## Benefits

- Grounded responses based on actual documents
- Reduced hallucination
- Up-to-date information
- Source citations
"""

# Save test file
test_path = "data/documents/test_doc.txt"
os.makedirs("data/documents", exist_ok=True)
with open(test_path, "w") as f:
    f.write(test_content)

# Load and chunk
docs = load_document(test_path)
print(f"Loaded {len(docs)} document(s)")

chunks = chunk_documents(docs)
print(f"Created {len(chunks)} chunk(s)")

for i, chunk in enumerate(chunks):
    print(f"\nChunk {i+1} ({len(chunk.page_content)} chars):")
    print(chunk.page_content[:100] + "...")

## 5. Test Vector Store

In [ ]:
from src.vectorstore import create_vectorstore, similarity_search

# Create vector store from chunks
vectorstore = create_vectorstore(chunks)
print("Vector store created!")

# Test similarity search
query = "What are the benefits of RAG?"
results = similarity_search(vectorstore, query, k=2)

print(f"\nSearch query: '{query}'")
print(f"Found {len(results)} results:")

for i, doc in enumerate(results):
    print(f"\nResult {i+1}:")
    print(doc.page_content[:200])

## 6. Test RAG Chain

In [ ]:
from src.chain import query_rag

# Test RAG query
question = "How does the RAG system work? Explain the steps."

result = query_rag(vectorstore, question, return_sources=True)

print("Question:", question)
print("\nAnswer:")
print(result["answer"])

print("\nSources:")
for source in result.get("sources", []):
    print(f"  - {source['source']} (Page {source['page']})")

## 7. Cleanup

In [ ]:
# Optional: Clean up test file
import os
if os.path.exists(test_path):
    os.remove(test_path)
    print(f"Removed test file: {test_path}")

---

## ✅ All Systems Ready!

If all cells executed successfully, your RAG system is properly configured and ready for use.

Next steps:
- Run `streamlit run app.py` to launch the web interface
- Upload your own documents and start asking questions!